In [1]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1Adeu5BVKa8nAqL22vlWDVUa25J49KYT2NLSg4e2RPhjm5tHjYxYs0byEQVo

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [17]:
Summer_UHI_yearly_1km=ee.Image('projects/ee-yceo/assets/Summer_UHI_yearly_300_v5')
Summer_Night2020_1km = Summer_UHI_yearly_1km.select(['summernighttime_UHI_2020'],['Summernighttime'])
Summer_Day2020_1km = Summer_UHI_yearly_1km.select(['summerdaytime_UHI_2020'],['Summerdaytime'])

In [18]:
urbext_2020 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2020')
urbext_data_2020 = geemap.ee_to_pandas(urbext_2020)

In [20]:
scale = 300

In [ ]:
nuhi_res = defaultdict(list)
duhi_res = defaultdict(list)
count_res = defaultdict(list)
for i in range(len(urbext_data_2020)):
    ua = urbext_data_2020.iloc[i]
    ua_f = urbext_2020.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
    geom = ua_f.geometry()
    nuhi_local_img = Summer_Night2020_1km.clip(geom)
    duhi_local_img = Summer_Day2020_1km.clip(geom)
    count_img = ee.Image(1).multiply(duhi_local_img.gt(-99999)).clip(geom)

    local_nuhi = nuhi_local_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('Summernighttime').getInfo()
    local_duhi = duhi_local_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('Summerdaytime').getInfo()
    local_count = count_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('constant').getInfo()
                
    nuhi_res[ua['country']].append(local_nuhi)
    duhi_res[ua['country']].append(local_duhi)
    count_res[ua['country']].append(local_count)
    
    print(i, end=' ')
    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 

In [ ]:
out_dicts = []
country_list = list(uhi_res.keys())
country_list.sort()
for country in country_list:
    out_dicts.append(
        {
            'country': country,
            'mean_summer_nighttime_uhi': sum(nuhi_res[country]) / sum(count_res[country]),
            'mean_summer_daytime_uhi': sum(duhi_res[country]) / sum(count_res[country]),
        }
    )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('CTY-11_uhi.csv')